In [ ]:
import pandas as pd
df = pd.read_csv('synthetic.csv')
df

,Attr_A,Attr_B,Attr_C,Attr_D,Attr_E,Attr_F,Attr_G,Attr_H,Attr_I,Attr_J,Attr_K,Attr_L,Attr_M,Attr_N,Class
0,12.478064,14.784992,1247.541877,100.962061,52.462177,1089.398211,10.575834,8.375407,10.288159,110.746551,994.367610,9.069350,1027.953917,109.672758,1
1,8.264345,8.854181,1389.686814,99.526529,65.106526,1186.523399,9.500485,10.088058,9.371983,78.210274,943.089589,9.988919,1120.317724,83.498764,3
2,9.875571,13.292442,779.077401,123.993772,104.699796,1201.722480,9.545266,14.266238,9.703551,86.252483,1082.989190,10.084217,970.953682,93.557046,2
3,9.207661,9.346913,965.468523,89.176009,102.628284,743.913507,9.777953,11.613946,8.912059,96.727873,812.800511,8.621781,947.207195,120.890054,3
4,8.863842,12.542969,1096.386230,106.595385,131.813380,883.059615,10.092974,13.556029,11.649982,21.566576,971.083175,10.072271,1007.583900,149.511979,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2293,8.537905,9.384823,1327.744849,105.043442,109.078735,1160.837366,10.054976,6.461691,9.718114,146.371553,1057.578998,9.504676,896.830168,50.119576,1
2294,10.727881,12.758686,1017.495424,103.388471,65.953337,955.265960,11.040854,5.930806,9.690326,92.318732,960.311986,9.262439,1158.151512,110.854719,1
2295,9.501612,12.449207,934.067397,94.681564,108.414931,1097.411669,11.379111,12.178924,9.161475,63.980205,972.994510,7.685836,996.569164,89.850887,0
2296,11.581467,12.024253,1247.894013,95.732086,85.265126,1012.536672,9.972115,9.001799,11.909924,128.966934,927.437658,8.906923,891.006210,93.561794,1


In [ ]:
#recherce des doublons de données pour le netoyage de code
bool_series = df.duplicated(keep='first')
print('Boolean series:')
compteur = 0
for i in bool_series.tolist():
  if i:
    compteur+=1
print(compteur)
#aucun doublons trouvé

Boolean series:
0


In [160]:
#preparation des données
indice_division = round(df.shape[0]*0.8)
jeu_entrainement = df[:indice_division]
jeu_test = df[indice_division:].reset_index()

In [ ]:
import math as m
def entropie(d):
  sum=0
  nbElements = d.shape [0] 
  series = d['Class'].value_counts()
  for i in series:
    proba = i/nbElements
    sum += proba * m.log2(proba)
  return -sum

In [ ]:
def gain(df, a, split_value) : 
  taille = df.shape[0]
  g = entropie(df)
  partitions = [None,None]
  sorted_data = df.sort_values(by = a, ignore_index=True)
  for i in range(0,sorted_data.shape[0]):
    if sorted_data[a][i] > split_value : 
      partitions[0] = sorted_data[:i]
      partitions[1] = sorted_data[i:]
      for i in range(2):
        g -= (len(partitions[i])/taille) * entropie(partitions[i])
      break
  return (g,partitions)

In [ ]:
class Noeud(object):
  def __init__(self,prediction, attribut=None, nfd = None, nfg =None,split_value = None):
    self.attribut = attribut
    self.right= nfd
    self.left = nfg
    self.prediction = prediction
    self.split_value = split_value
  def feuille(self):
    return self.left is None and self.right is None
  def resultat(self):
    res = ''
    for att in self.prediction.index.tolist():
      res += "Classe {}: Count {} ".format(att,self.prediction[att])
    return res

In [ ]:
def Decision_Tree(df,attribut_restant,profondeur,seuil):
  g = 0
  attribut = None
  partitions = [[],[]]
  pred = df["Class"].value_counts()
  for a in attribut_restant:
    for i in range(3):
      if(i==0):
        quartile = df.quantile(q=0.25, axis=0, numeric_only=True, interpolation='linear')[a]
      elif(i==1):
        quartile = df.quantile(q=0.5, axis=0, numeric_only=True, interpolation='linear')[a]
      else:
        quartile = df.quantile(q=0.75, axis=0, numeric_only=True, interpolation='linear')[a]
      (gainTemp,partitionsTemp) = gain(df,a,quartile)
      if(gainTemp > g):
        g = gainTemp
        attribut = a
        split_value = quartile
        partitions = partitionsTemp
  if attribut != None:
    attribut_restant.remove(attribut)
  if attribut_restant == None or profondeur > seuil:
    return Noeud(pred)
  if len(partitions[0])==0 or len(partitions[1])==0:
    return Noeud(pred)
  branche_gauche = Decision_Tree(partitions[0],attribut_restant,profondeur + 1,seuil)
  branche_droite = Decision_Tree(partitions[1],attribut_restant,profondeur + 1,seuil)
  return Noeud(pred,attribut,branche_droite,branche_gauche,split_value)

In [ ]:
def print_tree(node, spacing=''):
  if node is None:
    return
  if node.feuille():
    print(spacing + node.resultat())
    return
  print('{}[Attribute: {} Split value: {}]'.format(spacing,node.attribut,node.split_value))

  print(spacing + '> True')
  print_tree(node.left, spacing + '-')

  print(spacing + '> False')
  print_tree(node.right, spacing + '-')
  return


In [159]:
r = jeu_entrainement.columns.tolist()
r.remove("Class")
t = Decision_Tree(jeu_entrainement,r,0,4)
print_tree(t)

[Attribute: Attr_B Split value: 10.216836895342983]
> True
-[Attribute: Attr_F Split value: 1035.4338134218754]
-> True
--[Attribute: Attr_N Split value: 104.5007830967732]
--> True
---[Attribute: Attr_I Split value: 10.77284833630826]
---> True
----[Attribute: Attr_C Split value: 947.9993718600016]
----> True
-----Classe 0: Count 56 Classe 2: Count 6 Classe 1: Count 3 
----> False
-----Classe 0: Count 168 Classe 3: Count 15 Classe 1: Count 9 Classe 2: Count 2 
---> False
----[Attribute: Attr_H Split value: 10.510256053432833]
----> True
-----Classe 0: Count 24 Classe 1: Count 19 
----> False
-----Classe 0: Count 37 Classe 1: Count 4 Classe 2: Count 2 
--> False
---[Attribute: Attr_E Split value: 110.72247622659496]
---> True
----[Attribute: Attr_D Split value: 101.4879276334654]
----> True
-----Classe 1: Count 16 Classe 3: Count 10 Classe 0: Count 2 Classe 2: Count 1 
----> False
-----Classe 3: Count 14 Classe 1: Count 8 Classe 2: Count 4 Classe 0: Count 3 
---> False
----Classe 0: Co

In [169]:
#fonction qui teste une ligne du dataframe et retourne sa prediction
def test(arbre,donnee):
  if arbre.feuille():
    d = arbre.prediction.tolist()
    d.sort()
    for i in arbre.prediction.index.tolist():
      if arbre.prediction[i] == d[len(d)-1]:
        return i
  if(donnee[arbre.attribut] >= arbre.split_value):
    return test(arbre.right,donnee)
  else:
    return test(arbre.left,donnee)

In [171]:
def recuperer_prediction(arbre,df):
  res = []
  for _,i in df.iterrows():
    res.append(test(arbre,i))
  d = pd.DataFrame(res, columns =['pred'])
  return d 

recuperer_prediction(t,jeu_test)

,pred
0,0
1,0
2,1
3,1
4,1
...,...
455,1
456,1
457,1
458,1
